In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import max_error, r2_score, mean_absolute_error
import mlflow
import os

In [4]:
"""
Descarga de Archivos desde Google Drive sin Autenticación

Este script permite descargar archivos específicos desde Google Drive directamente a una ubicación local,
utilizando el ID de Google Drive del archivo (el archivo debe ser accesible para cualquier persona que tenga la URL).

El script procesa un archivo de texto ('archivos_info.txt') que contiene las IDs de Google Drive de los archivos,
los nombres con los que se desean guardar localmente, y las rutas locales de almacenamiento, todo separado por comas.

Formato esperado de 'archivos_info.txt':
ID_de_Google_Drive,Nombre_Archivo_Local,Ruta_Local

Ejemplo:
1hGvKmNAkK...,mi_archivo.txt,./descargas

Requisitos:
- Módulo 'requests' instalado en el entorno Python donde se ejecute este script.



"""

import requests
import os


def descargar_archivo_directo(id_archivo, directorio_destino, archivo_destino):
    """
    Descarga un archivo directamente desde Google Drive y lo guarda localmente.

    Parámetros:
    - id_archivo (str): ID del archivo en Google Drive.
    - directorio_destino (str): Ruta del directorio local donde se guardará el archivo.
    - archivo_destino (str): Nombre deseado para el archivo en local

    Devuelve:
    - archivo_destino (str): Nombre del archivo guardado.
    - ruta_completa (str): Ruta completa del archivo guardado.
    """
    # Construye la URL de descarga directa utilizando el ID del archivo
    url = f"https://drive.google.com/uc?export=download&id={id_archivo}"

    # Realiza la petición HTTP GET para descargar el archivo
    respuesta = requests.get(url, allow_redirects=True)

    # Comprueba que el directorio destino existe, si no, lo crea
    os.makedirs(directorio_destino, exist_ok=True)

    # Construye la ruta completa donde se guardará el archivo en local
    ruta_completa = os.path.join(directorio_destino, archivo_destino)

    # Guarda el contenido del archivo descargado en local
    with open(ruta_completa, 'wb') as archivo:
        archivo.write(respuesta.content)


    return archivo_destino, ruta_completa


def procesar_archivo_info(ruta_archivo_info):
    """
    Procesa un archivo de texto que contiene información sobre los archivos a descargar.

    Parámetros:
    - ruta_archivo_info (str): Ruta del archivo de texto que contiene los IDs de Google Drive,
                               los nombres de los archivos locales y las rutas locales.

    Devuelve:
    - Una lista de tuplas con el ID de Google Drive, el nombre local del archivo, y la ruta local.
    """
    archivos_info = []
    with open(ruta_archivo_info, 'r') as archivo:
        for linea in archivo:
            id_archivo, nombre_archivo, directorio_destino = linea.strip().split(',')
            archivos_info.append((id_archivo, nombre_archivo, directorio_destino))
    return archivos_info


In [5]:
# Ruta al archivo que contiene la información de los archivos a descargar
ruta_archivo_info = "..\info_archivos_GDrive_solo_entr.txt"

# Obtenemos la lista con la info de los archivos del fichero
archivos_a_descargar = procesar_archivo_info(ruta_archivo_info)

# Descargamos cada archivo de la lista
for id_archivo, nombre_archivo, directorio_destino in archivos_a_descargar:
    nombre_archivo_descargado, ruta_archivo_guardado = descargar_archivo_directo(id_archivo, directorio_destino, nombre_archivo)
    print(f"Archivo {nombre_archivo_descargado} guardado en: {ruta_archivo_guardado}")

Archivo datos_preparados_solo_entr.parquet guardado en: ../Downloads\datos_preparados_solo_entr.parquet


In [6]:
df = pd.read_parquet(ruta_archivo_guardado, engine='pyarrow')
df.describe()

,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,Tackles,...,nationality_The Gambia,nationality_Türkiye,nationality_Ukraine,nationality_United States,nationality_Uruguay,nationality_Venezuela,nationality_Wales,nationality_Zambia,nationality_Zimbabwe,marketValue
count,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,...,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2.251000e+03
mean,1271.695691,1.564638,1.063527,3.069303,0.168814,0.629276,76.128387,0.946246,0.622834,0.882719,...,0.000444,0.002665,0.001333,0.003554,0.028432,0.003998,0.000888,0.000444,0.000444,8.160329e+06
std,969.241058,2.906800,1.737932,2.958017,0.430942,0.604061,13.308598,0.962542,1.244505,0.693640,...,0.021077,0.051571,0.036490,0.059522,0.166240,0.063119,0.029801,0.021077,0.021077,1.188624e+07
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.500000e+04
25%,360.500000,0.000000,0.000000,1.000000,0.000000,0.200000,70.950000,0.300000,0.000000,0.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000e+06
50%,1157.000000,0.000000,0.000000,2.000000,0.000000,0.500000,78.300000,0.700000,0.000000,0.800000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000e+06
75%,2040.500000,2.000000,2.000000,5.000000,0.000000,0.900000,84.200000,1.300000,1.000000,1.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.500000e+06
max,3420.000000,27.000000,16.000000,15.000000,3.000000,4.500000,100.000000,8.700000,13.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.500000e+07


In [7]:
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]

# Partición de datos
RANDOM_STATE = 83
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_STATE)

In [8]:
# Establecer la URI de la base de datos SQLite y configuración de MLflow
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("redes_neuronales")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1713175269356, experiment_id='2', last_update_time=1713175269356, lifecycle_stage='active', name='redes_neuronales', tags={'mlflow.sharedViewState.35fcf56d61b8dbb98512f6829913993e41257533dd332e1e8ff07999f73a21ea': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Source`","attributes.`Models`","attributes.`Dataset`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","compareRunCharts":[{"uuid":"1713177259273xowwzda7","type":"BAR","runsCountToCompare":10,"metricSectionId":"171317725927396vp3vfh","deleted":false,"isGenerated":true,"metricKey":"best_mean_test_accuracy"},{"uuid":"1713177259273vnosmd9g","type":"BAR","runsCountToCompare":10,"metri

# Escalado | Con Hiperparámetros

In [9]:
with mlflow.start_run(run_name="evaluación_modelo_escalado_hiperparametros"):
    # Escalado de características
    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE, 
                    solver = 'adam',
                    alpha=0.01, batch_size=32,
                    learning_rate_init= 0.001,
                       hidden_layer_sizes=(24,))
                       
    scores = cross_validate(clf, scaled_X_train, y_train, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)
    train_accuracy = -np.mean(scores['train_score'])
    test_accuracy = -np.mean(scores['test_score'])

    clf.fit(scaled_X_train, y_train)

    test = clf.predict(scaled_X_test)

    print("mae:",mean_absolute_error(y_test, test),"\nmaximo error:", max_error(y_test, test),"\nr2",r2_score(y_test, test))

    mlflow.log_param("solver", 'adam')
    mlflow.log_param("alpha", 0.01)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("learning_rate_init", 0.001)
    mlflow.log_param("hidden_layer_sizes", (24,))
    mlflow.log_metric("mae", mean_absolute_error(y_test, test))
    mlflow.log_metric("r2", r2_score(y_test, test))

C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn

mae: 8147477.505081777 
maximo error: 69950600.59277306 
r2 -0.4849612322110435


C:\Users\Usuario\Documents\Alvaro\Carrera\2-Segundo\Primer cuatrimetre\Fundamentos de la Inteligencia Artificial\PycharmProjects\proyecto\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# Escalado | Default

In [10]:
with mlflow.start_run(run_name="evaluación_modelo_escalado_default"):
    # Escalado de características
    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_test = scaler.transform(X_test)


    scalery = StandardScaler()
    y_train_2d = y_train.values.reshape(-1,1)
    scaled_y_train = scalery.fit_transform(y_train_2d)
    y_train_final = scaled_y_train.ravel()

    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE)
                       
    scores = cross_validate(clf, scaled_X_train, y_train_final, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)
    train_accuracy = -np.mean(scores['train_score'])
    test_accuracy = -np.mean(scores['test_score'])

    clf.fit(scaled_X_train, y_train_final)

    test = clf.predict(scaled_X_test)
    test = test.reshape(-1, 1)
    test = scalery.inverse_transform(test)
    test= test.ravel()

    print("mae:",mean_absolute_error(y_test, test),"\nmaximo error:", max_error(y_test, test),"\nr2",r2_score(y_test, test))

    mlflow.log_metric("mae", mean_absolute_error(y_test, test))
    mlflow.log_metric("r2", r2_score(y_test, test))

mae: 3795093.9632694116 
maximo error: 33409805.030590747 
r2 0.7578740943492716


# Sin Escalar | Con Hiperparámetros

In [13]:
# Iniciar run de MLflow
with mlflow.start_run(run_name="MODELO FINAL"):
    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE,solver = 'adam',
                    alpha=0.01, batch_size=32,
                    learning_rate_init= 0.001,
                       hidden_layer_sizes=(24,))
    scores = cross_validate(clf, X_train, y_train, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)
    train_accuracy = -np.mean(scores['train_score'])
    test_accuracy = -np.mean(scores['test_score'])

    clf.fit(X_train, y_train)

    test= clf.predict(X_test)

    print("mae:",mean_absolute_error(y_test, test),"\nmaximo error:", max_error(y_test, test),"\nr2",r2_score(y_test, test))
    mlflow.log_param("solver", 'adam')
    mlflow.log_param("alpha", 0.01)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("learning_rate_init", 0.001)
    mlflow.log_param("hidden_layer_sizes", (24,))
    mlflow.log_metric("mae", mean_absolute_error(y_test, test))
    mlflow.log_metric("r2", r2_score(y_test, test))

mae: 2881621.90269101 
maximo error: 46492933.130645275 
r2 0.7580204829554815


# Sin Escalar | Default

In [14]:
# Iniciar run de MLflow
with mlflow.start_run(run_name="evaluación_modelo_no_escalado_default"):
    # Entrenamiento del modelo
    clf = MLPRegressor(random_state=RANDOM_STATE)
    scores = cross_validate(clf, X_train, y_train, scoring='neg_mean_absolute_error', cv=10, return_train_score=True)
    train_accuracy = -np.mean(scores['train_score'])
    test_accuracy = -np.mean(scores['test_score'])

    clf.fit(X_train, y_train)

    test= clf.predict(X_test)

    print("mae:",mean_absolute_error(y_test, test),"\nmaximo error:", max_error(y_test, test),"\nr2",r2_score(y_test, test))

    mlflow.log_metric("mae", mean_absolute_error(y_test, test))
    mlflow.log_metric("r2", r2_score(y_test, test))

2738098.4878543736 mae: 2910841.2629312365 
maximo error: 46491848.40584585 
r2 0.7545950526339287
